In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import cv2

In [18]:
print(os.listdir('D:\ILBS_14_8_21_Sanjeev\ILBS_Cropped_Volumes'+'\00910'))

ValueError: listdir: embedded null character in path

In [125]:
data_folder = os.listdir(r'D:\ILBS_14_8_21_Sanjeev\ILBS_Cropped_Volumes')

In [87]:
all_image = []
path = 'D:\\ILBS_14_8_21_Sanjeev\\ILBS_Cropped_Volumes'
for folder in data_folder:
    images = []
    IMG_HEIGHT = IMG_WIDTH =150
    for row in os.listdir(path+'\\'+folder):
        fpath = path+'\\'+folder+'\\'+row
        #print(fpath)
        image= cv2.imread( fpath, cv2.IMREAD_GRAYSCALE)
        image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
        image=np.array(image)
        image = image.astype('float32')
        image /= 255
        #image = np.expand_dims(image, axis=0)
        images.append(image)
    all_image.append(np.array(images))

In [88]:
len(all_image)

99

In [89]:
merged_data = np.array(all_image)

In [91]:
merged_data[0].shape

(150, 150, 150)

In [95]:
train_x = merged_data[0:85]

In [96]:
test_x = merged_data[85::]

In [238]:
test_x[0].shape

(150, 150, 150)

In [237]:
train_x[0].shape

(150, 150, 150)

In [101]:
train_y = data_folder[0:85]

In [102]:
test_y = data_folder[85::]

In [105]:
df_f = pd.read_excel('D:/ILBS_14_8_21_Sanjeev/Work_to_be_Completed_ILBS/HVPG value with CT images.xlsx',engine='openpyxl')

In [106]:
df_f.head()

,ID,Date,HVPG
0,9109,2019-01-30,21
1,19699,2019-02-13,12
2,31322,2019-02-15,17
3,45341,2019-02-13,16
4,49565,2019-02-06,10


In [128]:
labels = []
for row in df_f.values.tolist():
    #print(row[0])
    if row[0]=='9109':
        row[0]='00910'
    else:
        row[0]=row[0]
    #print(row[0])
    for folder in data_folder:
        #print((row[0]))
        if str(row[0])==str(folder):
            labels.append([folder,row[-1]])

In [130]:
df_labels = pd.DataFrame(labels,columns=['path','label'])

In [131]:
df_labels.head()

,path,label
0,9109,21
1,19699,12
2,31322,17
3,45341,16
4,49565,10


In [133]:
train_y = np.array(df_labels['label'][0:85])

In [134]:
test_y = np.array(df_labels['label'][85::])

In [138]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(85,)
(85,)
(14,)
(14,)


In [141]:
train_x[0].shape

(150, 150, 150)

In [157]:
test_x[0].shape

(150, 150, 150)

In [145]:
tr = [np.array(x) for x in train_x]

In [149]:
tr = [np.array(x) for x in train_x]
train_x_t = np.array(tr)
te = [np.array(z) for z in test_x]
test_x_t = np.array(te)

In [235]:
tr2 = [np.array(x) for x in train_x]
train_x_t2 = np.array(tr2)

In [231]:
te = [np.array(z) for z in test_x]
test_x_t = np.array(te)

In [242]:
type(test_x_t)

numpy.ndarray

In [233]:
X_train_f = np.expand_dims(train_x_t,axis=4)
#X_test_f = np.expand_dims(test_x_t, axis=4)

In [200]:
#y_train = np.array(df_train['label'])
y_train = keras.utils.to_categorical(train_y)
y_test = keras.utils.to_categorical(test_y)

In [176]:
X_train_f.shape

(85, 150, 150, 150, 1)

In [159]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, MaxPool2D
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf

In [184]:
from tensorflow import keras
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', 
                 input_shape=(150,150,150)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(28, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

print(model.summary())

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 148, 148, 32)      43232     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 146, 146, 64)      18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 341056)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               43655296  
_________________________________________________________________
dense_11 (Dense)             (None, 28)                3612      
Total params: 43,720,636
Trainable params: 43,720,636
Non-trainable params: 0
_________________________________________

In [189]:
history = model.fit(train_x_t, y_train,validation_split=0.1, batch_size=5, epochs=200, verbose=1)

Train on 76 samples, validate on 9 samples
Epoch 1/200
76/76 [==============================] - 1s 12ms/sample - loss: 2.0078 - accuracy: 0.5132 - val_loss: 3.0022 - val_accuracy: 0.1111
Epoch 2/200
76/76 [==============================] - 1s 12ms/sample - loss: 1.9451 - accuracy: 0.5395 - val_loss: 3.0294 - val_accuracy: 0.1111
Epoch 3/200
76/76 [==============================] - 1s 11ms/sample - loss: 1.9080 - accuracy: 0.5526 - val_loss: 3.0682 - val_accuracy: 0.1111
Epoch 4/200
76/76 [==============================] - 1s 11ms/sample - loss: 1.8574 - accuracy: 0.6711 - val_loss: 3.0923 - val_accuracy: 0.1111
Epoch 5/200
76/76 [==============================] - 1s 11ms/sample - loss: 1.7840 - accuracy: 0.6579 - val_loss: 3.0094 - val_accuracy: 0.1111
Epoch 6/200
76/76 [==============================] - 1s 11ms/sample - loss: 1.7457 - accuracy: 0.7237 - val_loss: 3.0360 - val_accuracy: 0.2222
Epoch 7/200
76/76 [==============================] - 1s 11ms/sample - loss: 1.6882 - accuracy

76/76 [==============================] - 1s 11ms/sample - loss: 0.3033 - accuracy: 0.9868 - val_loss: 3.4910 - val_accuracy: 0.1111
Epoch 57/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.2928 - accuracy: 0.9868 - val_loss: 3.5293 - val_accuracy: 0.1111
Epoch 58/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.2794 - accuracy: 1.0000 - val_loss: 3.5483 - val_accuracy: 0.1111
Epoch 59/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.2690 - accuracy: 0.9737 - val_loss: 3.4573 - val_accuracy: 0.1111
Epoch 60/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.2582 - accuracy: 0.9868 - val_loss: 3.4680 - val_accuracy: 0.1111
Epoch 61/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.2508 - accuracy: 0.9737 - val_loss: 3.4910 - val_accuracy: 0.1111
Epoch 62/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.2401 - accuracy: 0.9737 - val_loss: 3.5480 - val_accuracy: 0.111

76/76 [==============================] - 1s 11ms/sample - loss: 0.0585 - accuracy: 0.9868 - val_loss: 3.9113 - val_accuracy: 0.0000e+00
Epoch 113/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.0579 - accuracy: 0.9868 - val_loss: 3.8899 - val_accuracy: 0.1111
Epoch 114/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.0571 - accuracy: 0.9868 - val_loss: 3.9159 - val_accuracy: 0.1111
Epoch 115/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.0549 - accuracy: 0.9868 - val_loss: 3.9648 - val_accuracy: 0.1111
Epoch 116/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.0562 - accuracy: 0.9868 - val_loss: 4.0010 - val_accuracy: 0.0000e+00
Epoch 117/200
76/76 [==============================] - 1s 12ms/sample - loss: 0.0548 - accuracy: 0.9868 - val_loss: 3.9321 - val_accuracy: 0.0000e+00
Epoch 118/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.0533 - accuracy: 0.9868 - val_loss: 3.9466 - v

76/76 [==============================] - 1s 11ms/sample - loss: 0.0246 - accuracy: 1.0000 - val_loss: 4.2203 - val_accuracy: 0.1111
Epoch 168/200
76/76 [==============================] - 1s 10ms/sample - loss: 0.0288 - accuracy: 0.9868 - val_loss: 4.2227 - val_accuracy: 0.1111
Epoch 169/200
76/76 [==============================] - 1s 10ms/sample - loss: 0.0268 - accuracy: 0.9868 - val_loss: 4.2278 - val_accuracy: 0.1111
Epoch 170/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.0253 - accuracy: 1.0000 - val_loss: 4.2715 - val_accuracy: 0.1111
Epoch 171/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.0252 - accuracy: 1.0000 - val_loss: 4.2654 - val_accuracy: 0.1111
Epoch 172/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.0258 - accuracy: 1.0000 - val_loss: 4.2526 - val_accuracy: 0.1111
Epoch 173/200
76/76 [==============================] - 1s 11ms/sample - loss: 0.0229 - accuracy: 1.0000 - val_loss: 4.2710 - val_accuracy:

In [186]:
train_y.shape

(85,)

In [198]:
from tensorflow import keras
from tensorflow.keras.regularizers import l2
model_1 = Sequential()
model_1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', 
                 input_shape=(150,150,150)))
model_1.add(BatchNormalization())
model_1.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling2D(pool_size=(2,2)))
model_1.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
model_1.add(BatchNormalization())
model_1.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling2D(pool_size=(2,2)))
model_1.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling2D(pool_size=(2,2)))
model_1.add(Flatten())
# Densely connected layers
model_1.add(Dense(256, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001), bias_regularizer=l2(0.001)))
model_1.add(BatchNormalization())
model_1.add(Dropout(.5))
model_1.add(Dense(128, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001), bias_regularizer=l2(0.001)))
model_1.add(Dropout(.25))
model_1.add(Dense(28, activation='softmax'))

model_1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

print(model_1.summary())

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 148, 148, 32)      43232     
_________________________________________________________________
batch_normalization_23 (Batc (None, 148, 148, 32)      128       
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 148, 148, 32)      9248      
_________________________________________________________________
batch_normalization_24 (Batc (None, 148, 148, 32)      128       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 74, 74, 64)        18496     
_________________________________________________________________
batch_normalization_25 (Batc (None, 74, 74, 64)      

In [199]:
history = model_1.fit(train_x_t, y_train,validation_split=0.1, batch_size=5, epochs=200, verbose=1)

Train on 76 samples, validate on 9 samples
Epoch 1/200
76/76 [==============================] - 2s 23ms/sample - loss: 4.9736 - accuracy: 0.0263 - val_loss: 4.0065 - val_accuracy: 0.1111
Epoch 2/200
76/76 [==============================] - 1s 12ms/sample - loss: 4.7772 - accuracy: 0.0263 - val_loss: 4.0220 - val_accuracy: 0.1111
Epoch 3/200
76/76 [==============================] - 1s 12ms/sample - loss: 5.0656 - accuracy: 0.0395 - val_loss: 4.0404 - val_accuracy: 0.1111
Epoch 4/200
76/76 [==============================] - 1s 12ms/sample - loss: 4.8141 - accuracy: 0.0526 - val_loss: 4.0523 - val_accuracy: 0.1111
Epoch 5/200
76/76 [==============================] - 1s 12ms/sample - loss: 4.7160 - accuracy: 0.0395 - val_loss: 4.0664 - val_accuracy: 0.1111
Epoch 6/200
76/76 [==============================] - 1s 12ms/sample - loss: 4.6332 - accuracy: 0.0789 - val_loss: 4.0710 - val_accuracy: 0.1111
Epoch 7/200
76/76 [==============================] - 1s 12ms/sample - loss: 4.8656 - accuracy

76/76 [==============================] - 1s 12ms/sample - loss: 3.1583 - accuracy: 0.2632 - val_loss: 4.4813 - val_accuracy: 0.0000e+00
Epoch 57/200
76/76 [==============================] - 1s 12ms/sample - loss: 3.3680 - accuracy: 0.1974 - val_loss: 4.4725 - val_accuracy: 0.0000e+00
Epoch 58/200
76/76 [==============================] - 1s 11ms/sample - loss: 3.1598 - accuracy: 0.2632 - val_loss: 4.4571 - val_accuracy: 0.0000e+00
Epoch 59/200
76/76 [==============================] - 1s 12ms/sample - loss: 2.9490 - accuracy: 0.3158 - val_loss: 4.4474 - val_accuracy: 0.0000e+00
Epoch 60/200
76/76 [==============================] - 1s 11ms/sample - loss: 2.8523 - accuracy: 0.3684 - val_loss: 4.4601 - val_accuracy: 0.0000e+00
Epoch 61/200
76/76 [==============================] - 1s 12ms/sample - loss: 3.2591 - accuracy: 0.2763 - val_loss: 4.4824 - val_accuracy: 0.1111
Epoch 62/200
76/76 [==============================] - 1s 12ms/sample - loss: 2.9236 - accuracy: 0.3684 - val_loss: 4.4734 -

Epoch 111/200
76/76 [==============================] - 1s 12ms/sample - loss: 2.3314 - accuracy: 0.4868 - val_loss: 4.5488 - val_accuracy: 0.0000e+00
Epoch 112/200
76/76 [==============================] - 1s 12ms/sample - loss: 2.5224 - accuracy: 0.4605 - val_loss: 4.5550 - val_accuracy: 0.0000e+00
Epoch 113/200
76/76 [==============================] - 1s 11ms/sample - loss: 2.2845 - accuracy: 0.5526 - val_loss: 4.5902 - val_accuracy: 0.0000e+00
Epoch 114/200
76/76 [==============================] - 1s 12ms/sample - loss: 2.2838 - accuracy: 0.5921 - val_loss: 4.5746 - val_accuracy: 0.0000e+00
Epoch 115/200
76/76 [==============================] - 1s 11ms/sample - loss: 2.3226 - accuracy: 0.5658 - val_loss: 4.5776 - val_accuracy: 0.0000e+00
Epoch 116/200
76/76 [==============================] - 1s 11ms/sample - loss: 2.4325 - accuracy: 0.4868 - val_loss: 4.5516 - val_accuracy: 0.0000e+00
Epoch 117/200
76/76 [==============================] - 1s 11ms/sample - loss: 2.5404 - accuracy: 0.5

76/76 [==============================] - 1s 12ms/sample - loss: 1.8956 - accuracy: 0.6974 - val_loss: 4.6240 - val_accuracy: 0.0000e+00
Epoch 166/200
76/76 [==============================] - 1s 12ms/sample - loss: 1.9921 - accuracy: 0.6842 - val_loss: 4.5908 - val_accuracy: 0.0000e+00
Epoch 167/200
76/76 [==============================] - 1s 11ms/sample - loss: 2.0111 - accuracy: 0.6316 - val_loss: 4.5823 - val_accuracy: 0.0000e+00
Epoch 168/200
76/76 [==============================] - 1s 12ms/sample - loss: 1.9024 - accuracy: 0.6974 - val_loss: 4.5765 - val_accuracy: 0.1111
Epoch 169/200
76/76 [==============================] - 1s 12ms/sample - loss: 1.9292 - accuracy: 0.6579 - val_loss: 4.5567 - val_accuracy: 0.0000e+00
Epoch 170/200
76/76 [==============================] - 1s 12ms/sample - loss: 2.0100 - accuracy: 0.6974 - val_loss: 4.5569 - val_accuracy: 0.1111
Epoch 171/200
76/76 [==============================] - 1s 12ms/sample - loss: 1.9950 - accuracy: 0.6842 - val_loss: 4.5765

In [ ]:
te = [np.array(z) for p in test_x]
test_x_t = np.array(te)

In [207]:
tr = [np.array(x) for x in train_x]
train_x_t = np.array(tr)

(14,)

In [223]:
# score = model_1.evaluate(test_x_t, y_test, verbose = 1) 

# print('Test loss:', score[0]) 
# print('Test accuracy:', score[1])
score = model_1.evaluate(train_x_t, y_train, verbose = 1) 

print('Train loss:', score[0]) 
print('Train accuracy:', score[1])

85/85 [==============================] - 1s 15ms/sample - loss: 1.2284 - accuracy: 0.9059
Train loss: 1.2284007212694954
Train accuracy: 0.90588236


In [212]:
te = [np.array(p) for p in test_x]
test_x_t = np.array(te)

In [211]:
for i in test_x.shape

(14,)

In [224]:
test_x_t.shape

(14,)

In [225]:
train_x_t.shape

(85, 150, 150, 150)